<a href="https://colab.research.google.com/github/faithrts/Science_Explainers/blob/main/analysis/analysis_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
### importing libraries

# basic libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# to download files
from google.colab import files

import os
import re
import time
import math
import codecs

# for stats
from scipy.stats import chi2

# sklearn libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# NLTK
import nltk
from nltk import word_tokenize
from nltk import pos_tag
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Importing databases

In [3]:
### cloning git repos

!git clone https://github.com/faithrts/Science_Explainers
#!git clone https://github.com/dhmit/gender_novels
#!git clone https://github.com/faithrts/Short_Fiction

Cloning into 'Science_Explainers'...
remote: Enumerating objects: 1124, done.
remote: Total 1124 (delta 0), reused 0 (delta 0), pack-reused 1124
Receiving objects: 100% (1124/1124), 57.46 MiB | 17.09 MiB/s, done.
Resolving deltas: 100% (166/166), done.
Updating files: 100% (915/915), done.


In [4]:
### saving databases into dataframes

explainer_df = pd.read_csv('Science_Explainers/database/science_explainers_database.csv')
fiction_df = pd.read_csv('short_fiction_database.csv')

In [2]:
### unzipping short fiction files

!unzip txt_files.zip

Archive:  txt_files.zip
   creating: txt_files/
  inflating: __MACOSX/._txt_files    
   creating: txt_files/ATLANTIC/
  inflating: __MACOSX/txt_files/._ATLANTIC  
  inflating: txt_files/.DS_Store     
  inflating: __MACOSX/txt_files/._.DS_Store  
   creating: txt_files/PARIS_REVIEW/
  inflating: __MACOSX/txt_files/._PARIS_REVIEW  
   creating: txt_files/GRANTA/
  inflating: __MACOSX/txt_files/._GRANTA  
   creating: txt_files/NEW_YORKER/
  inflating: __MACOSX/txt_files/._NEW_YORKER  
   creating: txt_files/THE_SUN/
  inflating: __MACOSX/txt_files/._THE_SUN  
   creating: txt_files/TIN_HOUSE/
  inflating: __MACOSX/txt_files/._TIN_HOUSE  
   creating: txt_files/AGNI/
  inflating: __MACOSX/txt_files/._AGNI  
   creating: txt_files/ELECTRIC/
  inflating: __MACOSX/txt_files/._ELECTRIC  
   creating: txt_files/LIT_HUB/
  inflating: __MACOSX/txt_files/._LIT_HUB  
  inflating: txt_files/ATLANTIC/LamourCa.txt  
  inflating: __MACOSX/txt_files/ATLANTIC/._LamourCa.txt  
  inflating: txt_files/AT

## Helper functions

### Data processing

In [5]:
### turns all column names to upper case
def uppercase_columns(df):
  columns = df.columns
  new_columns = [column.upper() for column in columns]
  df.columns = new_columns

In [7]:
### counts the word count of the text and adds it as a column
def count_text_length(df):
  df['LENGTH'] = ''

  for index, row in df.iterrows():
    text = row['TEXT']
    text_length = len(text)
    row['LENGTH'] = text_length

In [6]:
def load_text_content(df, path):

  # adds new column to the dataframe
  df['TEXT'] = ''

  for index, row in df.iterrows():
    cur_filename = row['FILENAME']

    # renaming files with weird accent characters in their names
    if 'í' in cur_filename and os.path.isfile(path + cur_filename.replace('í', 'í')):
      os.rename(path + cur_filename.replace('í', 'í'), path + cur_filename)
    if 'é' in cur_filename and os.path.isfile(path + cur_filename.replace('é', 'é')):
      os.rename(path + cur_filename.replace('é', 'é'), path + cur_filename)

    cur_article = codecs.open(path + cur_filename, 'r', encoding = 'utf8').read()

    # saving the text in the dataframe
    df.at[index, 'TEXT'] = cur_article

  return df

In [8]:
### custom pre-processor to eliminte numbers and instances of "_", "\", and "—"
def my_preprocessor(text):
    text = text.lower()
    text = re.sub('([0-9—_\\\\])', '', text)
    return text

In [9]:
### makes all the column names UPPERCASE
def col_names_to_uppercase(df):
  new_columns = [name.upper() for name in df.columns]
  df.columns = new_columns

  return df

In [10]:
class StemWords(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass

  def fit(self, X, y=None):
    return self

  def transform(self, list_of_passages):
    # initializes the stemmer
    snowball_stemmer = SnowballStemmer('english')
    new_list_of_passages = []

    for passage in list_of_passages:
      # breaks the passage up into its component words
      words = nltk.word_tokenize(passage)
      new_words = [snowball_stemmer.stem(word) for word in words]

      new_passage = ' '.join(new_words)
      new_list_of_passages.append(new_passage)

    return new_list_of_passages

In [11]:
def refine_df_columns(list_of_titles, df):

  # the new df with only the columns to keep
  df_copy = df[list_of_titles]

  return df_copy

### Adding features to dataframes

In [12]:
def add_dtm(df, focus_col, keep_symbols = False):

 # using CountVectorizer to make a DTM based on the words in the corpus
  if keep_symbols:
    vectorizer = CountVectorizer(lowercase = False, token_pattern = '[A-Z]+\$*', min_df = 5)
  else:
    vectorizer = CountVectorizer(preprocessor = my_preprocessor, stop_words = 'english', min_df = 5)

  dtm = vectorizer.fit_transform(df[focus_col])
  words = vectorizer.get_feature_names_out()

  # converting sparse matrix to an array of arrays
  matrix = dtm.toarray()

  # combining the DTM with the metadata (associated word)
  DTM = pd.DataFrame(matrix, columns = words)

  # attaching the DTM to the original dataframe
  dtm_both = pd.concat([df, DTM], axis=1)

  return dtm_both

In [13]:
def add_tf_idf(df, focus_col):

  # using TfidfVectorizer to add the tf-idf values of each word to the dataframe
  vectorizer = TfidfVectorizer(preprocessor = my_preprocessor, stop_words = 'english', min_df = 5)

  tf_idf = vectorizer.fit_transform(df[focus_col])
  words = vectorizer.get_feature_names_out()

  # converting sparse matrix to an array of arrays
  matrix = tf_idf.toarray()

  # combining the tf-idf matrix with the metadata (associated words)
  TF_IDF = pd.DataFrame(matrix, columns = words)

  # attaches the tf-idf to the original dataframe
  tf_idf_both = pd.concat([df, TF_IDF], axis = 1)

  return tf_idf_both

In [14]:
### assumes the POS tags are in a column called 'POS TAGS'
def count_pos_tags(df):
  # concatenates all lists of POS tags into one big lists
  all_tags = df['POS TAGS'].sum()

  # counts each POS tag occurrence
  tag_counts = Counter(all_tags)

  # sorts the POS tags
  sorted_tag_counts = sorted(tag_counts, reverse = True)

  return tag_counts, sorted_tag_counts

In [15]:
### assumes the text content is in a column called 'TEXT'
def add_pos_tags(df):

  new_df = df.copy()

  new_df['POS TAG TOKENS'] = ''
  new_df['POS TAGS'] = ''
  new_df['POS TAGS STRING'] = ''

  for index, row in new_df.iterrows():
    cur_text = row['TEXT']
    tokenized_text = word_tokenize(cur_text)
    POS_tags = pos_tag(tokenized_text)
    tags_only = [tag for word,tag in POS_tags]

    row['POS TAG TOKENS'] = POS_tags
    row['POS TAGS'] = tags_only
    row['POS TAGS STRING'] = ' '.join(tags_only)

  return new_df

# Loading content

In [8]:
### adding the text of each article as a column in the dataframe

# science explainers
explainer_df = load_text_content(explainer_df, 'Science_Explainers/txt_files/')
count_text_length(explainer_df)

# fiction
fiction_df = load_text_content(fiction_df, 'txt_files/')
count_text_length(fiction_df)

## Extending dataframes

In [22]:
### adding the DTM to the dataframes

explainer_dtm_df = add_dtm(explainer_df, 'TEXT')
fiction_dtm_df = add_dtm(fiction_df, 'TEXT')

explainer_dtm_df = explainer_dtm_df.drop(columns = ['TEXT'])
fiction_dtm_df = fiction_dtm_df.drop(columns = ['TEXT'])

In [23]:
### adding the tf-idf values to the dataframes

explainer_tfidf_df = add_tf_idf(explainer_df, 'TEXT')
fiction_tfidf_df = add_tf_idf(fiction_df, 'TEXT')

explainer_tfidf_df = explainer_tfidf_df.drop(columns = ['TEXT'])
fiction_tfidf_df = fiction_tfidf_df.drop(columns = ['TEXT'])

In [24]:
### adding POS tags to the dataframes
explainer_pos_df = add_pos_tags(explainer_df)
fiction_pos_df = add_pos_tags(fiction_df)

explainer_pos_df = add_dtm(explainer_pos_df, 'POS TAGS STRING', keep_symbols = True)
fiction_pos_df = add_dtm(fiction_pos_df, 'POS TAGS STRING', keep_symbols = True)

explainer_pos_df = explainer_pos_df.drop(columns = ['TEXT'])
fiction_pos_df = fiction_pos_df.drop(columns = ['TEXT'])

# Downloading files

In [26]:
### downloading csv of dataframes

dfs_to_download = ['explainer_dtm_df', 'fiction_dtm_df',
                   'explainer_tfidf_df', 'fiction_tfidf_df',
                   'explainer_pos_df', 'fiction_pos_df']

for df_name in dfs_to_download:
  filename = df_name.split('_df')[0] + '.csv'
  eval(df_name).to_csv(filename, index = False, escapechar='\\')
  files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>